# Учебный проект "Исследование надежности заемщиков"


Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Цель и задачи проекта:

### Цель проекта:

Выявление зависимости погашения кредита в срок от семейного положения и количества детей у клиента банка


### Задачи проекта:

1. Импортировать все необходимые для исследования библиотеки, открыть dataset и изучить сохраненные в нем данные.

2. Предобработать представленные данные стандартными методами Python; выбрать данные, необходимые для исследования.

3. Категоризировать данные и сформулировать общий вывод по исследованию.

______

### Шаг 1. Открытие файла с данными и изучение общей информации 

In [1]:
#Прежде всего, загружаем все необходимые для исследования библиотеки

import pandas as pd
from pymystem3 import Mystem
from nltk.stem import SnowballStemmer 
from collections import Counter


In [2]:
#Читаем dataset с помощью метода .read_csv()
data=pd.read_csv('/datasets/data.csv')

In [3]:
#Выводим первые 15 значений, поскольку бывает, что в самом начале (5 - 10 первых значений)ошибок может не быть...
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
#При помощи .describe() можно получить общую информацию
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


#### Предварительные выводы:

1. В 'children' есть (-1) и (20). (-1) - отсутстсующая информация? (20) - либо ошибка, либо многодетная семья (и то, и другое вполне возможно).

2. В 'days_employed' - среднее значение равно (63046). 63046/365 = 172.7 года стажа (!). При этом минимальное значение стажа (-18388). Предполагаем, что это ошибки. Отрицательные значения можно изменить методом .abs().

3. В 'dob_years' присутствует нулевой возраст (0), скорее всего, тоже ошибка.

______

In [5]:
#Воспользуемся методами .info() и isnull().sum()
data.info()
data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

#### Предварительные выводы:

1. В таблице имеются данные, требующие предобработки: имеются пропуски и значения, для которых необходима замена (столбцы 'days_employed' и 'total_income'). 

2. В 'days_employmed' и 'total_income' отсутствуют значения (есть пропуски).

3. Также есть данные, требующие лемматизации.

<div class="alert alert-success">

**Комментарий ревьюера:** Хорошее ознакомление, тебе удалось сразу обнаружить все аномалии и пропуски в данных.
</div>

_____

### Дальнейшие действия:

1. Проверка соответствия пропущенных данных в столбцах 'days_employed' и 'total_income'.

2. Выяснение среднего реального трудового стажа.

3. Проверка корректности данных по детям.

4. Проверка корректности данных в столбце 'dob_years'.

_____

In [6]:
#Проверяем, соответствуют ли пропущенные данные в столбцах 'days_employed' и 'total_income'
data[(data['total_income'].isnull()==True)&(data['days_employed'].isnull()==True)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
dob_years           2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
debt                2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


#### Предположение подтвердилось: пропущенные данные в выбранных столбцах соответствуют 

______

In [7]:
#Проверяем принадлежность людей из разных сфер деятельности 
#Если все принадлежат одной, можно будет заполнить NaN средним по данной группе
data[(data['total_income'].isnull()==True)&(data['days_employed'].isnull()==True)]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

#### Увы, клиенты из разных сфер деятельности: NaN следует заполнять по среднему из каждой группы

______

In [8]:
#Выясняем количество строк в 'days_employed' > 0
print('Количество строк days_employed > 0:', data[data['days_employed']>0].shape[0])

#Выясняем количество строк в 'days_employed' < 0
print('Количество строк days_employed < 0:', data[data['days_employed']<0].shape[0])

#Выясняем количество пенсионеров в 'days_employed' > 0
data_ret=data[(data['days_employed']>0)&(data['income_type']=='пенсионер')]
print('Количество пенсионеров days_employed > 0:', data_ret.shape[0])

#Выясняем количество пенсионеров со стажем > 0 и количеством отработанных лет > 20
print('Количество пенсионеров days_employed >0 и стажем > 20 лет:', data_ret[data_ret['days_employed']>7300].shape[0])

#Выясняем средний стаж среди пенсионеров
print('Средний стаж среди пенсионеров (в годах):', data_ret['days_employed'].mean()/365)

Количество строк days_employed > 0: 3445
Количество строк days_employed < 0: 15906
Количество пенсионеров days_employed > 0: 3443
Количество пенсионеров days_employed >0 и стажем > 20 лет: 3443
Средний стаж среди пенсионеров (в годах): 1000.0095650544143


#### Результаты проверки гипотезы:
1. Большая часть значений в'days_employed', а точнее - 75% - отрицательные и не относятся к пенсионерам. Их можно заменять на значения по модулю, хотя нужно учитывать, что данный столбец для исследования фактически не имеет значения.
2. Остальные значения в 'days_employed', имеющие значения >0 - это пенсионеры (кроме двух), но при этом их средний стаж работы ничего общего с действительностью не имеет - налицо ошибка данных.

_____

In [9]:
#Проверяем корректность даннных в столце 'children'
print('Количество людей с числом детей -1:', data[data['children']==-1].count()[0])
print('Количество людей с числом детей 20:', data[data['children']==20].count()[0])

Количество людей с числом детей -1: 47
Количество людей с числом детей 20: 76


#### И то, и другое значние не уникальны, являются явными ошибками записи и могут быть исправлены

_____


In [10]:
#Проверяем нулевое значение в столбце 'dob_years'
print('Количество людей с нулевым возрастом:', data[data['dob_years']==0].count()[0])

Количество людей с нулевым возрастом: 101


#### 101 человек с нулевым возрастом - явно либо ошибка в записи (что маловероятно), либо просто не внесенные данные. Также нужно исправлять. 

____

### Шаг 1. Общие выводы:

1. Данные, содержащиеся в dataset имеют пропуски (NaN), требующие замены.
2. Часть данных является некорректной (отрицательные значения количества детей, средний стаж пенсионеров, люди с нулевым возрастом) и также требует замены.
3. Часть данных требует лемматизации.
 

___

### Дальнейшие действия:

1. Обработка пропусков: парные пропуски NaN в 'days_employed' и 'total_income' заполним по 'income_type' в соответствие с возрастом.
2. Замена типов данных: в 'days_employed' заменяем отрицательные значения по модулю. Данным в 'days_employed' и 'total_income' присваиваем целочисленные значения (для удобства). Приводим данные в 'children' к нормальным значениям. То же самое делаем с нулевым возрастом в 'dob_years' и со стажем > 90 лет в 'days_employed'.
3. Лемматизация данных: проведем лемматизацию в 'education' а также в 'purpose'.
4. Категоризация данных: создадим дополнительные категории в дополнение к уже существующим ('education_id'и 'family_status_id). Ими будут: категоризация по уровню зарплаты, по стажу работы и по количеству детей. 

____

<div class="alert alert-success">

**Комментарий ревьюера:** Все делаешь правильно, хороший исследовательский подход к анализу данных 👍
</div>

### Шаг 2. Предобработка данных

### Обработка пропусков

Сначала необходимо разобраться с отрицательным стажем и со средними значениями стажа в тысячу (!) лет.
Можно создать pivot_table по каждой из категорий в 'income_type', для того, чтобы понять, можно ли к 'days_employed' применять среднее и метод .abs().

In [11]:
#Группируем данные методом .groupby() и применяем метод .agg
data_grouped=data.groupby('income_type').agg({'days_employed':['count','mean',lambda x:sum(x>0)]})

#Создаем таблицу
dict_to_rename = dict(zip(data_grouped.columns.levels[1], ['Количество строк', 'Среднее', 'Значения больше нуля']))

#Выводим таблицу
data_grouped = data_grouped.rename(columns=dict_to_rename, level=1)
data_grouped

days_employed                                    
                Количество строк        Среднее Значения больше нуля
income_type                                                         
безработный                    2  366413.652744                  2.0
в декрете                      1   -3296.759962                  0.0
госслужащий                 1312   -3399.896902                  0.0
компаньон                   4577   -2111.524398                  0.0
пенсионер                   3443  365003.491245               3443.0
предприниматель                1    -520.848083                  0.0
сотрудник                  10014   -2326.499216                  0.0
студент                        1    -578.751554                  0.0

#### Все данные, кроме данных по типу "безработные" и "пенсионеры" однотипны. Следовательно, использовать среднее значение и метод .abs() можно.

___

### Дальнейшие действия: 

1. Используем метод .abs() для данных в 'days_employed' и 'total_income'.
2. Добавляем столбец 'ratio_employed' в котором будет отражен коэффициент количества отработанных дней (с учетом того, что банковкие операции можно начинать проводить с 18 лет). Этот коэффициент позволит заполнить NaN в 'days_employed'с учетом среднего показателя по группе.
3. Заполняем NaN в 'days_employed' заменой среднего по группе 'income_type', умноженного на количество дней.
4. Заполняем Nan в 'total_income' средним значением по зарплате в 'income_type'.



In [12]:
#Применяем метод .abs() для 'total_income' и 'days_employed'
data[['total_income', 'days_employed']] = data[['total_income', 'days_employed']].abs()

#Добавляем столбец 'ratio-employed', применяя метод .notnull() и учитывая возраст от 18 лет
data['ratio_employed'] = data[data['days_employed'].notnull()]['days_employed']/((data['dob_years']-18)*365)

#Выводим первые 15 значений 
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,ratio_employed
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,0.963205
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,0.612603
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,1.027109
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,0.807191
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,26.635309
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,0.281944
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,0.315529
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,0.013080
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,1.116820
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,0.260721


In [13]:
#Заполняем NaN в 'days_employed'
data['days_employed']=data.groupby('income_type')['days_employed'].transform(lambda x: x.fillna(x.mean()*data['dob_years']*365))

#Заполняем NaN в 'total_income'
data['total_income']=data.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.mean()))

#Выводим первые 15 строк
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,ratio_employed
0,1,8.437673e+03,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,0.963205
1,1,4.024804e+03,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,0.612603
2,0,5.623423e+03,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,1.027109
3,3,4.124747e+03,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,0.807191
4,0,3.402661e+05,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,26.635309
5,0,9.261858e+02,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,0.281944
6,0,2.879202e+03,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,0.315529
7,0,1.527796e+02,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,0.013080
8,2,6.929865e+03,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,1.116820
9,0,2.188756e+03,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,0.260721


#### Теперь можно удалить вспомогательный столбец 'ratio_employed' и вывести общую информацию по датасету

___

In [14]:
# Удаляем столбец 'ratio_emoloyed' методом .drop()
data=data.drop('ratio_employed',axis=1)

#Выводим data.info()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


___

#### Продолжаем обрабатывать некорректные значения:
1. Значения в'education' - приводим к единообразному виду.
2. Исправляем (-1) в 'children' на (0)
3. Исправляем нулевой возраст в 'dob_years'

In [15]:
#Приводим значения в 'education' к единому виду методом .str.lower()
data['education']=data['education'].str.lower()

#Выводим первые 15 строк
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8.437673e+03,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4.024804e+03,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5.623423e+03,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4.124747e+03,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,3.402661e+05,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,9.261858e+02,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2.879202e+03,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,1.527796e+02,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6.929865e+03,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2.188756e+03,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [16]:
#Изменям количество детей с (-1) на (0)
data['children']=data['children'].replace(-1,0)

#Проверяем данные по количеству детей
print('Количество детей = -1:', data[data['children']==-1].count()[0])
print('Количество детей = 0:', data[data['children']==0].count()[0])
print('Количество детей = 2:', data[data['children']==2].count()[0])
print('Количество детей >=20:', data[data['children']>=20].count()[0])

Количество детей = -1: 0
Количество детей = 0: 14196
Количество детей = 2: 2055
Количество детей >=20: 76


In [17]:
#Изменяем количество детей (20) на (2)
data['children']=data['children'].replace(20,2)

#Проверяем данные по количеству детей
print('Количество детей = 2:', data[data['children']==2].count()[0])

Количество детей = 2: 2131


#### Количество детей = 2 увеличилось на 76 (2055 + 76). Ошибка исправлена.

#### Гипотеза: возможно, нулевой возраст - это несовершеннолетние клиенты (<=18)? Стоит проверить... 

In [18]:
#Проверяем гипотезу о том, что (0) в 'dob_years' - это клиенты возрастом <= 18 лет
print(data[data['dob_years']==0].count()[0])
print(data[data['dob_years']<=18].count()[0])

101
101


#### В датасете индексом (0) обозначались клиенты, не достигшие 19 лет

В принципе, ничего не мешает заменить нулевые значения в 'dob_years' на средние по каждой группе

In [19]:
#Заменяем нулевые значения
data['dob_years']=data.groupby('income_type')['dob_years'].transform(lambda x:x.replace(0, int(x.mean())))
                                                                     
#Проверяем, остались ли нулевые значения в 'dob_years'
print('Количество нулевых значений в "dob_years":', data[data['dob_years']==0].count()[0])                                                                    
                                                                     

Количество нулевых значений в "dob_years": 0


#### Еще раз воспользуемся методами .info() и .describe(), для того, чтобы вывести общие сведения о датасете

In [20]:
#Выводим общую инофрмацию о таблице
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,2.152500e+04,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.477538,1.522834e+08,43.495238,0.817236,0.972544,0.080883,1.673959e+05
std,0.755467,1.078074e+09,12.230322,0.548138,1.420324,0.272661,9.790695e+04
min,0.000000,0.000000e+00,19.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,1.024652e+03,34.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,2.605748e+03,43.000000,1.000000,0.000000,0.000000,1.519313e+05
75%,1.000000,3.336411e+05,53.000000,1.000000,1.000000,0.000000,2.024175e+05
max,5.000000,9.725518e+09,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Вывод:

После применения стандартных методов выявления и обработки пропусков, dataset пропусков не имеет. Также, методом .abs() удалось привести данные в 'days_employed' и 'total_income' к объективно номальным значениям.

___

<div class="alert alert-success">

**Комментарий ревьюера:** Мне понравилось как ты разобрался с данными, особенно красиво ты заполнил пропуски в зависимости от принадлежности к группе. Видно, что ты хорошо овладел необходимыми инструментами :)
</div>

### Замена типа данных

#### Необходимо перевести   .float в .int. Для этой цели воспользуемся методом .astype(int)

In [21]:
#Заменяем .foat в 'days_employed'на .int
data['days_employed']=data['days_employed'].astype(int)

#Заменяем .foat в 'total_income'на .int
data['total_income']=data['total_income'].astype(int)

#Выводим данные
data.dtypes

children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

### Вывод:

Методом .astype() значения в столбцах 'days_employed' и 'total_income' были переведены в целое число (присвоено целочисленное значение).


#### Примечание: 
Метод pd.to_numeric() в данном случае не подходит, поскольку значения в данных столбцах уже изначально представляли собой float.

___

### Обработка дубликатов

In [22]:
#Находим дубликаты с помощью стандартного метода .duplicated()и суммируем их методом .sum()
data.duplicated().sum()

71

#### 71 дубликат.  Их следует удалить...

In [23]:
#Удаляем методом .drop_duplicates() и еще раз проверяем датасет на наличие методом .sum()
data= data.drop_duplicates()
data.duplicated().sum()

0

### Вывод:

В датафрейме присутствовали дубликаты. Они были удалены стандартными методами .duplicated() и .drop_duplicates()

___

### Лемматизация

#### В столбце 'purpose' содержится информация (object) с целями получения кредита. Эти данные должны быть подвергнуты лемматизации методом m.lemmatize(). Кроме того,  необходимо использовать метод .apply(), чтобы обработать весь столбец.

In [24]:
#Используем методы m.lemmatize() и .apply()
m=Mystem()
data['purpose_lemma'] = data['purpose'].apply(m.lemmatize)

#Выводим первые 15 строк
print(data['purpose_lemma'].head(15))

0                             [покупка,  , жилье, \n]
1                   [приобретение,  , автомобиль, \n]
2                             [покупка,  , жилье, \n]
3                [дополнительный,  , образование, \n]
4                           [сыграть,  , свадьба, \n]
5                             [покупка,  , жилье, \n]
6                      [операция,  , с,  , жилье, \n]
7                                   [образование, \n]
8                 [на,  , проведение,  , свадьба, \n]
9           [покупка,  , жилье,  , для,  , семья, \n]
10                     [покупка,  , недвижимость, \n]
11    [покупка,  , коммерческий,  , недвижимость, \n]
12                          [сыграть,  , свадьба, \n]
13                  [приобретение,  , автомобиль, \n]
14           [покупка,  , жилой,  , недвижимость, \n]
Name: purpose_lemma, dtype: object


#### Данные в столбце 'purpose' лемматизировались.

#### Теперь можно написать функцию, проводящую категоризацию имеющихся лемм:

In [25]:
#Пишем функцию категоризации лемм, используя только однозначные существительные
def purpose_category(lemmas):
    for lemma in lemmas:
        if ('жилье' in lemmas) or ('недвижимость' in lemmas):
            return 'Займ на покупку недвижимости'
        if ('образование' in lemmas):
            return 'Займ на образование'
        if ('автомобиль' in lemmas):
            return 'Займ на покупку автомобиля'
        if ('свадьба' in lemmas):
            return 'Займ на свадьбу'
        
#Применяем метод .apply()
data['category'] = data['purpose_lemma'].apply(purpose_category) 

#Выводим первые 15 строк
print(data['category'].head(15))

0     Займ на покупку недвижимости
1       Займ на покупку автомобиля
2     Займ на покупку недвижимости
3              Займ на образование
4                  Займ на свадьбу
5     Займ на покупку недвижимости
6     Займ на покупку недвижимости
7              Займ на образование
8                  Займ на свадьбу
9     Займ на покупку недвижимости
10    Займ на покупку недвижимости
11    Займ на покупку недвижимости
12                 Займ на свадьбу
13      Займ на покупку автомобиля
14    Займ на покупку недвижимости
Name: category, dtype: object


#### Проверим наличие столбца 'category' в датасете и корректность данных в нем:

In [26]:
#Выводим первые 15 строк общей таблицы
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma,category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",Займ на покупку недвижимости
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",Займ на покупку автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",Займ на покупку недвижимости
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",Займ на образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",Займ на свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",Займ на покупку недвижимости
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",Займ на покупку недвижимости
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",Займ на образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",Займ на свадьбу
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",Займ на покупку недвижимости


#### Теперь можно составить список всех лемм, для того, чтобы выбрать из него наиболее часто встречающиеся существительные. Это понадобится для дальнейшей категоризации запросов клиентов банка.

In [27]:
#Сначала просто создаем список уникальных запросов "Займ на..."
print(data['category'].value_counts())   

Займ на покупку недвижимости    10811
Займ на покупку автомобиля       4306
Займ на образование              4013
Займ на свадьбу                  2324
Name: category, dtype: int64


In [28]:
#А потом, создаем список уникальных лемм, основываясь на уже созданном столбце 'category'
m = Mystem()
list_of_lemmas = []
for element in data['category']:
    lemma = m.lemmatize(element)
    list_of_lemmas.extend(lemma)

unique_lemmas = Counter(list_of_lemmas)
sorted(unique_lemmas.items(), key = lambda pair: pair[1], reverse=True)

[(' ', 58025),
 ('заем', 21454),
 ('на', 21454),
 ('\n', 21454),
 ('покупка', 15117),
 ('недвижимость', 10811),
 ('автомобиль', 4306),
 ('образование', 4013),
 ('свадьба', 2324)]

#### Поскольку количество уникальных запросов "Займ на покупку..." и  количество лемм  "недвижимость", "автомобиль", "образование" и "свадьба" совпадают, данные леммы можно использовать для создания списка

#### Далее, с помощью метода стемминга получаем основы данных слов...

In [29]:
top_purposes=['недвижимость','автомобиль','образование','свадьба']

russian_stemmer = SnowballStemmer('russian')

stemmed_purposes = [russian_stemmer.stem(word) for word in top_purposes]
print(stemmed_purposes)

['недвижим', 'автомобил', 'образован', 'свадьб']


#### ...и присваиваем им соответствующие категории

In [30]:
dict_stemmed_purposes = {v:k for k,v in enumerate(stemmed_purposes)}
dict_stemmed_purposes['недвижим'] = 0
print(dict_stemmed_purposes)

{'недвижим': 0, 'автомобил': 1, 'образован': 2, 'свадьб': 3}


### Вывод:

Данные в столбце 'purpose' были лемматизированы. Результаты лемматизации отражены в столбце 'purpose_lemma'. Реализация функции позволила привести уникальные данные в столбце 'purpose' к четырем универсальным категориям целям заема: "Займ на покупку недвижимости", "Займ на покупку автомобиля", "Займ на свадьбу" и "Займ на образование".С помощью метода стемминга были получены основы четырех главных слов - запросов и на их основе был создан словарь списка целей вида "цель - номер цели".

___


### Категоризация данных

### Дальнейшие действия:
1. Создание дополнительного столбца с категориями из dict_stemmed_purposes.
2. Проверка корректности совпадений с парах с изначально присвоенными категориями.
3. Создание (разбиение) категорий по доходам, стажу, возрасту и количеству детей, реализация функций.
4. Сведение данных в общую таблицу. 

In [31]:
#Создаем дополнительный столбец 'cat_index'с категориями из dict_stemmed_purposes
data['cat_index'] = 0
for row in range(len(data)):
    for purpose in dict_stemmed_purposes:
        if purpose in data.iloc[row, 13]:
            data.iloc[row, 14] = dict_stemmed_purposes[purpose]
            
#Выводим первые 15 строк            
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma,category,cat_index
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",Займ на покупку недвижимости,0
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",Займ на покупку автомобиля,1
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",Займ на покупку недвижимости,0
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",Займ на образование,2
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",Займ на свадьбу,3
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",Займ на покупку недвижимости,0
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",Займ на покупку недвижимости,0
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",Займ на образование,2
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",Займ на свадьбу,3
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",Займ на покупку недвижимости,0


___

#### Проверяем корректность совпадений категорий в столбцах, для которых категории были заданы изначально: 'education' и 'education_id', а также 'family_status' и 'family_status_id'

In [32]:
#Корректность совпадений в паре 'education' - 'education_id'
data.groupby(['education', 'education_id']).size().to_frame('count')

,,count
education,education_id,
высшее,0,5250
начальное,3,282
неоконченное высшее,2,744
среднее,1,15172
ученая степень,4,6


#### Данные в паре 'education' и 'education_id' корректны и уникальны

In [33]:
#Корректность совпадений в паре 'family_status' - 'family_status_id'
data.groupby(['family_status', 'family_status_id']).size().to_frame('count')

,,count
family_status,family_status_id,
Не женат / не замужем,4,2810
в разводе,3,1195
вдовец / вдова,2,959
гражданский брак,1,4151
женат / замужем,0,12339


#### Данные в паре 'family_status' и 'family_status_id' корректны и уникальны

___

#### Теперь стоит сгруппировать клиентов банка в 3 - 4 группы исходя из дохода, стажа, возраста и количества детей. Для этого можно воспользоваться методом .describe(), который показывает разбиение по процентам: 25%, 50%, 75%. Возможно, это поможет в процессе группировки.

In [34]:
#Выводим сведения по датасету, применяя метод .describe()
data.describe()


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income,cat_index
count,21454.000000,2.145400e+04,21454.000000,21454.000000,21454.000000,21454.000000,2.145400e+04,21454.000000
mean,0.478372,1.424659e+08,43.473758,0.817097,0.973898,0.081150,1.674316e+05,0.899786
std,0.756009,1.041747e+09,12.225661,0.548674,1.421567,0.273072,9.806060e+04,1.055592
min,0.000000,0.000000e+00,19.000000,0.000000,0.000000,0.000000,2.066700e+04,0.000000
25%,0.000000,1.020000e+03,33.000000,1.000000,0.000000,0.000000,1.076230e+05,0.000000
50%,0.000000,2.588000e+03,42.000000,1.000000,0.000000,0.000000,1.518870e+05,0.000000
75%,1.000000,3.325852e+05,53.000000,1.000000,1.000000,0.000000,2.024170e+05,2.000000
max,5.000000,9.725518e+09,75.000000,4.000000,4.000000,1.000000,2.265604e+06,3.000000


#### Наиболее очевидно группирование по возрасту и количеству детей:

#### По возрасту 'dob_years':
- до 30 лет
- от 30 до 45 лет
- от 45 до 65 лет
- свыше 65 лет

#### По количеству детей 'children':
- нет детей
- 1 - 2 ребенка
- 3 - 4 ребенка
- 5 и больше детей

#### По продолжительности стажа 'days_employed':
- менее 10 лет (меньше 3650 дней)
- от 10 до 30 лет (меньше 10950 дней)
- от 30 до 45 лет (меньше 16 425 дней)
- свыше 45 лет

#### Для большей наглядности применим методы .min(), mean(), max() для столбца 'total_income'

In [35]:
#Для столбца 'days_employed'
print(data['total_income'].min())
print(data['total_income'].mean())
print(data['total_income'].max())

20667
167431.58371399273
2265604


#### Полученные значения, в принципе, можно применять для создания групп. Однако, было бы уместно использовать более широкий диапазон значений, который максимально соответствует реальности...

#### По доходу 'total_income':
- малообеспеченный (меньше 50.000)
- средний класс (от 50.000 до 120.000)
- обеспеченный (от 120.000 до 1.000.000)
- богатый (свыше 1.000.000)


___

#### Создаем функции для новых категорий с учетом формирования групп по стажу, доходу, количеству детей и возрасту:

In [36]:
#Создаем функции
def dob_years_index(row):
    if row['dob_years']<30:
        return 'до 30 лет'
    if row['dob_years']<45:
        return 'от 30 до 45 лет'
    if row['dob_years']<65:
        return 'от 45 до 65 лет'
    else:
        return 'свыше 65 лет'   
    
def children_index(row):
    if row ['children']==0:
        return 'нет детей'
    if row['children']<=2:
        return '1 - 2 ребенка'
    if row['children']<=4:
        return '3 - 4 ребенка'
    else:
        return '5 и больше детей'
    
def days_emlpoyed_index(row):
    if row['days_employed']<3650:
        return 'менее 10 лет'
    if row['days_employed']<10950:
        return 'от 10 до 30 лет'
    if row['days_employed']<16425:
        return 'от 30 до 45 лет'
    else:
        return 'свыше 45 лет'
    
def total_income_index(row):
    if row['total_income']<50000:
        return 'малообеспеченный'
    elif 50000<row['total_income']<120000:
        return 'средний класс'
    elif 120000<row['total_income']<=1000000:
        return 'обеспеченный'
    else:
        return 'богатый'


In [37]:
#Добавляем новые столбцы в свобдную таблицу
data['dob_years_index']=data.apply(dob_years_index, axis=1)
data['children_index']=data.apply(children_index, axis=1)
data['days_emlpoyed_index']=data.apply(days_emlpoyed_index, axis=1)
data['total_income_index']=data.apply(total_income_index,axis=1)

In [38]:
#Выводим первые 15 строк новой таблицы
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma,category,cat_index,dob_years_index,children_index,days_emlpoyed_index,total_income_index
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",Займ на покупку недвижимости,0,от 30 до 45 лет,1 - 2 ребенка,от 10 до 30 лет,обеспеченный
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",Займ на покупку автомобиля,1,от 30 до 45 лет,1 - 2 ребенка,от 10 до 30 лет,средний класс
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",Займ на покупку недвижимости,0,от 30 до 45 лет,нет детей,от 10 до 30 лет,обеспеченный
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",Займ на образование,2,от 30 до 45 лет,3 - 4 ребенка,от 10 до 30 лет,обеспеченный
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",Займ на свадьбу,3,от 45 до 65 лет,нет детей,свыше 45 лет,обеспеченный
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",Займ на покупку недвижимости,0,до 30 лет,нет детей,менее 10 лет,обеспеченный
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",Займ на покупку недвижимости,0,от 30 до 45 лет,нет детей,менее 10 лет,обеспеченный
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",Займ на образование,2,от 45 до 65 лет,нет детей,менее 10 лет,обеспеченный
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",Займ на свадьбу,3,от 30 до 45 лет,1 - 2 ребенка,от 10 до 30 лет,средний класс
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",Займ на покупку недвижимости,0,от 30 до 45 лет,нет детей,менее 10 лет,обеспеченный


### Вывод:

#### Данные категоризированы по:
- 'education' (категории были присвоены изначально)
- 'family_status_id' (категории были присвоены изначально)
- 'days_employed'
- 'total_income'
- 'dob_years'
- 'children'

Проведена проверка корректности данных в столбцах с изначально присвоенными категориями.

<div class="alert alert-success">

**Комментарий ревьюера:** Тут ты даже перевыполнил план и категоризовал все что только можно было :) 
</div>

### Шаг 3. Ответьте на вопросы

### Дальнейшие действия:
- Создаем функцию для выявления зависимостей.
- С помощью данной функции получаем ответы на поставленные вопросы.
- Проверяем данные из функции с помощью сводной таблицы, созданной методом .pivot_table().
- Формулируем окончательный вывод.

In [39]:
#Создаем функцию для выявления зависимостей
def correlation(category):
    return data.groupby(category)['debt'].mean().to_frame().sort_values(by='debt')

___

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [40]:
correlation('children_index')

,debt
children_index,
5 и больше детей,0.000000
нет детей,0.075258
3 - 4 ребенка,0.083558
1 - 2 ребенка,0.093137


In [41]:
#Создаем таблицу с помощью метода .pivot_table()
data_pivot=data.pivot_table(index=['children_index'],columns='category',values='debt',aggfunc='mean')

#Выводим первые 15 строк таблицы
data_pivot.head(15)

category,Займ на образование,Займ на покупку автомобиля,Займ на покупку недвижимости,Займ на свадьбу
children_index,,,,
1 - 2 ребенка,0.106698,0.111030,0.082433,0.087302
3 - 4 ребенка,0.052632,0.085714,0.084211,0.142857
5 и больше детей,0.000000,0.000000,0.000000,0.000000
нет детей,0.086350,0.085434,0.067061,0.075065


### Вывод:

Максимально высокий риск невозврата кредита в срок демонстрируют заемщики, имеющие 1 - 2 детей.

___

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [42]:
correlation('family_status')

,debt
family_status,
вдовец / вдова,0.065693
в разводе,0.071130
женат / замужем,0.075452
гражданский брак,0.093471
Не женат / не замужем,0.097509


In [43]:
#Создаем таблицу с помощью метода .pivot_table()
data_pivot=data.pivot_table(index=['family_status'],columns='category',values='debt',aggfunc='mean')

#Выводим первые 15 строк таблицы
data_pivot.head(15)

category,Займ на образование,Займ на покупку автомобиля,Займ на покупку недвижимости,Займ на свадьбу
family_status,,,,
Не женат / не замужем,0.107452,0.128728,0.081454,NaN
в разводе,0.071429,0.074733,0.069527,NaN
вдовец / вдова,0.075377,0.091743,0.051661,NaN
гражданский брак,0.148515,0.117512,0.092012,0.080034
женат / замужем,0.083237,0.083699,0.069349,NaN


### Вывод:

Не женатые / не замужние заемщики, а также заемщики, живущие гражданским браком, чаще других категорий не возвращают кредиты в срок.

___

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [44]:
correlation('total_income_index')

,debt
total_income_index,
малообеспеченный,0.061828
богатый,0.080000
обеспеченный,0.081459
средний класс,0.081570


In [45]:
#Создаем таблицу с помощью метода .pivot_table()
data_pivot=data.pivot_table(index=['total_income_index'],columns='category',values='debt',aggfunc='mean')

#Выводим первые 15 строк таблицы
data_pivot.head(15)

category,Займ на образование,Займ на покупку автомобиля,Займ на покупку недвижимости,Займ на свадьбу
total_income_index,,,,
богатый,0.250000,0.000000,0.058824,0.000000
малообеспеченный,0.024691,0.135135,0.044693,0.078947
обеспеченный,0.089332,0.094801,0.074390,0.076031
средний класс,0.102543,0.088578,0.069243,0.088798


### Вывод:

Малообеспеченные слои населения менее склонны к тому, чтобы не возвращать кредиты в срок. При этом представители среднего класса, а также обеспеченные клиенты банка чаще других задерживают выплаты по кредитам.

- Как разные цели кредита влияют на его возврат в срок?

In [46]:
correlation('cat_index')

,debt
cat_index,
0,0.072334
3,0.080034
2,0.092200
1,0.093590


In [47]:
#Создаем таблицу с помощью метода .pivot_table()
data_pivot=data.pivot_table(index=['cat_index'],columns='category',values='debt',aggfunc='mean')

#Выводим первые 15 строк таблицы
data_pivot.head(15)

category,Займ на образование,Займ на покупку автомобиля,Займ на покупку недвижимости,Займ на свадьбу
cat_index,,,,
0,NaN,NaN,0.072334,NaN
1,NaN,0.09359,NaN,NaN
2,0.0922,NaN,NaN,NaN
3,NaN,NaN,NaN,0.080034


### Вывод:

Заемщики, приобретающие недвижимость максимально ответственны с возвратом кредита в срок; клиенты банка, приобретающие автомобили, напротив, склонны к просрочкам по кредиту.

____

#### Формируем финальную сводную таблицу с помощью метода .pivot_table() для 'family_status' и 'children_index'.

In [48]:
#Создаем таблицу с помощью метода .pivot_table()
data_pivot=data.pivot_table(index=['family_status', 'children_index'],columns='category',values='debt',aggfunc='mean')

#Выводим первые 15 строк таблицы
data_pivot.head(15)

category                                Займ на образование  \
family_status         children_index                          
Не женат / не замужем 1 - 2 ребенка                0.094828   
                      3 - 4 ребенка                0.000000   
                      нет детей                    0.111354   
в разводе             1 - 2 ребенка                0.050000   
                      3 - 4 ребенка                     NaN   
                      нет детей                    0.082278   
вдовец / вдова        1 - 2 ребенка                0.157895   
                      3 - 4 ребенка                0.000000   
                      нет детей                    0.067797   
гражданский брак      1 - 2 ребенка                0.187500   
                      3 - 4 ребенка                0.166667   
                      5 и больше детей                  NaN   
                      нет детей                    0.129630   
женат / замужем       1 - 2 ребенка                0.100956   
                      3 - 4 ребенка                0.046875   

category                                Займ на покупку автомобиля  \
family_status         children_index                                 
Не женат / не замужем 1 - 2 ребенка                       0.157895   
                      3 - 4 ребенка                       0.000000   
                      нет детей                           0.123314   
в разводе             1 - 2 ребенка                       0.097561   
                      3 - 4 ребенка                       0.000000   
                      нет детей                           0.065990   
вдовец / вдова        1 - 2 ребенка                       0.058824   
                      3 - 4 ребенка                            NaN   
                      нет детей                           0.094527   
гражданский брак      1 - 2 ребенка                       0.146853   
                      3 - 4 ребенка                       0.333333   
                      5 и больше детей                    0.000000   
                      нет детей                           0.098592   
женат / замужем       1 - 2 ребенка                       0.102740   
                      3 - 4 ребенка                       0.068966   

category                                Займ на покупку недвижимости  \
family_status         children_index                                   
Не женат / не замужем 1 - 2 ребенка                         0.108911   
                      3 - 4 ребенка                         0.666667   
                      нет детей                             0.073643   
в разводе             1 - 2 ребенка                         0.072961   
                      3 - 4 ребенка                         0.100000   
                      нет детей                             0.066975   
вдовец / вдова        1 - 2 ребенка                         0.092308   
                      3 - 4 ребенка                         0.000000   
                      нет детей                             0.046512   
гражданский брак      1 - 2 ребенка                         0.123839   
                      3 - 4 ребенка                         0.000000   
                      5 и больше детей                           NaN   
                      нет детей                             0.078582   
женат / замужем       1 - 2 ребенка                         0.074788   
                      3 - 4 ребенка                         0.083333   

category                                Займ на свадьбу  
family_status         children_index                     
Не женат / не замужем 1 - 2 ребенка                 NaN  
                      3 - 4 ребенка                 NaN  
                      нет детей                     NaN  
в разводе             1 - 2 ребенка                 NaN  
                      3 - 4 ребенка                 NaN  
                      нет детей                     NaN  
вдовец / вдова        1 - 2 ребенка             

<div class="alert alert-success">

**Комментарий ревьюера:** И с этим этапом ты справился мастерски :) Особенно приглянулось как ты считаешь процент невозвратов с помощью `mean()` - интересный подход :)
</div>

### Шаг 4. Общий вывод:

- Наиболее проблемными клиентами являются заемщики, не состоящие в официальных отношениях (гражданский брак/ не женат/ не замужем) и имеющие при этом 1 - 2 детей.
- Менее рискованными с точки зрения возврата кредита являются клиенты состоящие в браке и также имеющие 1 - 2 детей.
- Наиболее ответственными заемщиками являются клиенты со статусом "в разводе", "вдовец / вдова", а также состоящие в гражданском браке, не имеющие детей.
- Малоимущие клиенты стараются оплачивать кредит в срок, в то время как клиенты, относящиеся в категориям "средний класс" и "обеспеченные", склонны к просрочке платежей по кредитам.
- Наиболее аккуратно и ответственно к кредитам относятся клиенты, берущие займ на приобретение недвижимости; клиенты, берущие деньги на покупку автомобиля - менее ответственны.

<div class="alert alert-success">

**Вывод:** Ты проделал большую работу и у тебя получился совершенный и аккуратный проект! Редко бывает, когда проект принимается первого раза, а у тебя это получилось! Так что ты вдвойне молодец! :) Поздравляю с завершением своего первого проекта и желаю дальнейших успехов! :)  
</div>

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.